Calibrage modèle multi-design

In [ ]:

case = 0 # nb_case = len(stepConditions)

# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])])

# Dictionnaire des paramètres du modèle
model_parameters = dict([
    ('tau_g', X['componentSpecs']['pv']['tau_g']), # Transmittance glass
    ('alpha_g', X['componentSpecs']['pv']['alpha_g']), # Absorptance glass
    ('eps_PV', X['componentSpecs']['pv']['eps_PV']), # Émissivité cellule PV
    ('eps_g', X['componentSpecs']['pv']['eps_g']), # Émissivité glass
    ('eps_hx_top', X['componentSpecs']['main']['eps_hx_top']), # Émissivité de l'échangeur thermique sur le panneau
    ('eps_hx_back', X['componentSpecs']['main']['eps_hx_back']), # Émissivité de l'échangeur thermique à l’arrière du panneau
    ('eps_ins', X['componentSpecs']['main']['eps_ins']), # Émissivité isolant
    ('lambd_air',  X['componentSpecs']['main']['lambd_air']) # Épaisseur de la couche d'air
    ])

